In [77]:
import numpy as np
from hmm_utils import HMM
from attackers.smoothing_state_attacker import ss_attacker
from attackers.decoding_attacker import dec_attacker
from params import *
from solvers.mc_ennumeration import MC_enumeration
#from solvers.simulated_annealing import simulated_annealing
#from solvers.aps_gibbs import aps_gibbs
from solvers.nn_RS.nn_RS import *
#from solvers.aps_gibbs_class import *

from solvers.aps_gibbs_class_exp_decay import *
from solvers.random_mc_ennumeration import *

from scipy.special import softmax

def ohe(x, n_obs):
    return np.diag(np.ones(n_obs))[ x.astype(int) ]

In [78]:
data = np.load("data/HMM_structure_experiment_comb3_X_30_T_10_Q_10.npy", allow_pickle=True).tolist()
priors     = data["prior"]
transition = data["trans"]
emission   = data["emiss"]
rho_probs = 1.0*np.ones(30)
x_vector = data["X"]

In [79]:
hmm = HMM(n_components=10, n_obs=30 )
hmm.emissionprob_ = emission
hmm.transmat_     = transition
hmm.startprob_    = priors


In [80]:
V, seq = hmm.nu(x_vector)
seq

array([4, 5, 4, 2, 6, 7, 4, 0, 1, 2])

In [81]:
attacker = dec_attacker(priors, transition, emission, rho_probs,
         x_vector, w1 = 10.0 ,w2 = 0.0 , seq = 0*np.ones(10, dtype='int'), k_value=10000)

# Random MC

In [82]:
rmc = random_mc_ennumeration(attacker)

In [83]:
z_star, quality = rmc.iterate(simulation_seconds=5)

In [84]:
quality

121.86579762319757

In [85]:
attack_obs = attacker.attack_X(np.ones_like(z_star), z_star)
V, seq = hmm.nu(attack_obs.astype(int))
seq

array([8, 0, 7, 5, 4, 0, 0, 9, 9, 1])

# APS

In [86]:
from solvers.aps_gibbs_class import aps_gibbs
aps = aps_gibbs(attacker, np.arange(500, 10000, 10))

In [87]:
z_star, quality = aps.iterate(simulation_seconds=5)

In [88]:
quality

158.44692562050014

In [89]:
attack_obs = attacker.attack_X(np.ones_like(z_star), z_star)
V, seq = hmm.nu(attack_obs.astype(int))
seq

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])